In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

df = pd.read_csv('/kaggle/input/indoor-localization-using-ble-and-wifi/BLE_rssi_after_FFT_Filter_Grid_1.5m_2.5m.csv')
# df = pd.read_csv('BLE_rssi_V12.csv')
# df = pd.read_csv('Data/BLE_rssi_after_FFT_Filter_Grid_1.5m_2.5m.csv')

In [ ]:
df['location'] = df['location'].replace(np.unique(df['location'].to_numpy()), np.arange(55))

In [ ]:
one_hot = pd.get_dummies(df['location'])

In [ ]:
import tensorflow as tf

In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.LSTM(150, input_shape=[5, 5]))
model.add(tf.keras.layers.Dense(55, activation='softmax'))
# model.add(tf.keras.layers.LSTM(55, input_shape=[5, 5]))
# model.add(tf.keras.layers.Dense(55, activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam',  metrics=['accuracy'])

In [ ]:
x = df[['beacon1', 'beacon2', 'beacon3', 'beacon4', 'beacon5']].to_numpy()
x = x[:6350].reshape([-1, 5, 5])

In [ ]:
y = one_hot[:6350].to_numpy()
y = y.reshape([-1, 5, 55])
y = np.sum(y, 1)
# for item in y:
#     print(item)
# print(y[0])
# print(np.argmax(y[0]))
# print(y)
# for item in y:
#     print(item)
for label in y:
    max_value_index = np.argmax(label)
    label[max_value_index] = 1
    label[0:max_value_index] = 0
    label[max_value_index + 1:] = 0
    
# print('*******************************************************')
# for item in y:
#     print(item)
# y[y > 1] = 1

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)
print(X_train.shape)
print(y_train.shape)

In [ ]:
history = model.fit(X_train, y_train, epochs=430)

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
model.summary()

In [ ]:
fig = plt.figure()
plt.plot(history.history['loss'],'r')
plt.plot(history.history['accuracy'],'b')
plt.title('model accuracy and loss per epoch')
plt.ylabel('accuracy and loss')
plt.xlabel('epoch')
plt.legend(['loss', 'accuracy'], loc='upper right')
plt.grid()

In [ ]:
#save entire model to HDF5 file
model.save('LSTM_saved_model.h5')
#Recreate the exact same model, including weights and optimizer
new_model = tf.keras.models.load_model('LSTM_saved_model.h5')
model.summary()